# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 4. Klasifikasi Naive Bayes


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-03-25 16:04:08--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-03-25 16:04:08 (49.1 MB/s) - ‘iris.csv’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [3]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [4]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data, test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [5]:
print(data_uji.shape[0], data_latih.shape[0])

30 120


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_uji**

In [6]:
label_uji = data_uji.pop('variety')

In [7]:
data_uji.head()

,sepal.length,sepal.width,petal.length,petal.width
126,6.2,2.8,4.8,1.8
99,5.7,2.8,4.1,1.3
112,6.8,3.0,5.5,2.1
15,5.7,4.4,1.5,0.4
6,4.6,3.4,1.4,0.3


## 3) Menghitung Prior

Tahapan pertama pada algoritma Naive Bayes adalah perhitungan prior. Prior suatu kelas merupakan peluang awal munculnya kelas tersebut. Atau dengan kata lain, prior merupakan frekuensi relatif dari suatu kelas terhadap keseluruhan data.

Buatlah fungsi bernama **hitung_prior** yang berfungsi menghitung nilai prior dari masing-masing kelas yang terdapat pada data latih. Fungsi ini menerima input berupa list kelas dari data latih.

In [8]:
from collections import Counter
def hitung_prior(list_kelas):
  n_data = len(list_kelas)
  prior = Counter(list_kelas)
  for key in prior.keys():
    prior[key] = prior[key] / n_data
  return prior

Ujilah fungsi **hitung_prior** menggunakan kelas dari variabel **data_latih**. Amati hasilnya

In [9]:
prior = hitung_prior(data_latih['variety'])
print(prior)

Counter({'Setosa': 0.3333333333333333, 'Virginica': 0.3333333333333333, 'Versicolor': 0.3333333333333333})


## 4) Menghitung *likelihood*

Praktikum kali ini mengimplementasikan metode Naive Bayes menggunakan datset iris. Semua fitur pada dataset iris memiliki tipe numerik. Dengan demikian, perhitungan *likelihood* dilakukan menggunakan Gaussian Model.

Tahapan yang perlu dilakukan sebelum perhitungan *likelihood* adalah menghitung rata-rata dan deviasi standar masing-masing fitur per kelas. Buatlah fungsi **hitung_rata2_std_kelas** yang berfungsi menghitung rata-rata dan standar deviasi per fitur dan kelas

In [33]:
def hitung_rata2_std_kelas(input_data):
  list_columns = input_data.columns[:-1]
  class_column_name = input_data.columns[-1]
  list_class = set(input_data[class_column_name])
  rata2 = {}
  std = {}
  for column in list_columns:
    for a_class in list_class:
      rata2[(a_class, column)]= input_data.loc[input_data[class_column_name] == a_class][column].mean()
      std[(a_class, column)] = input_data.loc[input_data[class_column_name] == a_class][column].std()
  return(rata2, std)

Buatlah fungsi **hitung_likelihood_gaussian** yang bertujuan menghitung nilai likelihood suatu nilai terhadap rata-rata dan deviasi standar tertentu.

In [34]:
import math
def hitung_likelihood_gaussian(data,rata2,std):
  hasil = (1/math.sqrt(2*math.pi*(std**2)))*math.exp((-1*((data-rata2)**2))/(2*(std**2)))
  return hasil

## 5) Proses *training*
Proses training pada Naive Bayes dengan model Gaussian dilakukan untuk menghitung prior serta  rata-rata dan deviasi standar pada masing-msaing fitur dan kelas.

In [35]:
def training_naive_bayes_gaussian(data_latih):
  class_column_name = data_latih.columns[-1]
  prior = hitung_prior(data_latih[class_column_name])
  (rata2, std) = hitung_rata2_std_kelas(data_latih)
  list_class = set(data_latih[class_column_name])
  list_columns = data_latih.columns[:-1]
  model = {}
  model['prior'] = prior
  model['rata2'] = rata2
  model['std'] = std
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

Lakukanlah proses training dan simpan hasil training pada variabel **model**

In [36]:
model = training_naive_bayes_gaussian(data_latih)

In [37]:
print(model)

{'prior': Counter({'Setosa': 0.3333333333333333, 'Virginica': 0.3333333333333333, 'Versicolor': 0.3333333333333333}), 'rata2': {('Virginica', 'sepal.length'): 6.6625, ('Versicolor', 'sepal.length'): 5.9275, ('Setosa', 'sepal.length'): 5.015000000000001, ('Virginica', 'sepal.width'): 2.9899999999999998, ('Versicolor', 'sepal.width'): 2.7675, ('Setosa', 'sepal.width'): 3.41, ('Virginica', 'petal.length'): 5.612499999999999, ('Versicolor', 'petal.length'): 4.235, ('Setosa', 'petal.length'): 1.4525000000000001, ('Virginica', 'petal.width'): 2.0324999999999998, ('Versicolor', 'petal.width'): 1.3199999999999998, ('Setosa', 'petal.width'): 0.2375}, 'std': {('Virginica', 'sepal.length'): 0.612974044709219, ('Versicolor', 'sepal.length'): 0.5267157555831107, ('Setosa', 'sepal.length'): 0.34310049361367684, ('Virginica', 'sepal.width'): 0.3128036985813209, ('Versicolor', 'sepal.width'): 0.33922044870631374, ('Setosa', 'sepal.width'): 0.3657097069310435, ('Virginica', 'petal.length'): 0.520939732

## 6) Proses *testing*
Proses testing dilakukan dengan menghitung nilai posterior dari data uji berdasarkan nilai model yang diperoleh saat training. Nilai posterior diperoleh dari perkalian prior dengan *likelihood* masing-masing fiturnya. Setelah itu, penentuan kelas data uji dilakukan berdasarkan nilai posterior terbesar

In [38]:
def testing_naive_bayes_gaussian(model, data_uji):
  prior = model['prior']
  rata2 = model['rata2']
  std = model['std']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class, 1)
  for a_class in list_class:
    for column in list_columns:
      posterior[a_class] = posterior[a_class] * hitung_likelihood_gaussian(data_uji[column], rata2 [(a_class, column)], std[(a_class, column)])
    posterior[a_class] = posterior[a_class] * prior[a_class]
  kelas_uji = max(posterior, key = posterior.get)
  return kelas_uji

Ujilah fungsi **testing_naive_bayes_gaussian** menggunakan data uji pertama. Bandingkan hasilnya dengan nilai label sebenarnya.

In [39]:
indeks_uji = 0
prediksi = testing_naive_bayes_gaussian(model,data_uji.iloc[indeks_uji])
print(prediksi)
print(label_uji.iloc[indeks_uji])

Virginica
Virginica


Lakukan pengujian untuk semua data uji. Selanjutnya bandingkan dengan label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar

In [40]:
prediksi_total = []
for indeks in range (data_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_gaussian(model, data_uji.iloc[indeks]))
print("Total prediksi benar: ", sum(prediksi_total == label_uji))

Total prediksi benar:  28


## TUGAS
Pada tugas kali ini Anda akan menggunakan dataset **[Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)** yang telah dimodifikasi. Fitur-fitur yang ada bertipe kategori sebagai berikut:
1. **buying**. Merupakan harga beli mobil. Nilai : vhigh, high, med, low
2. **maint**. Menandakan biaya perawatan mobil. Nilai: vhigh, high, med, low
3. **lug_boot**. Menandakan ukuran bagasi. Nilai: small, med, big
4. **safety**. Menandakan skor keamanan mobil. Nilai:  low, med, high
5. **class**. Merupakan kelas data. Nilai: unacc, acc, good, vgood

Download dataset dari https://dataset-ppm.s3.amazonaws.com/car_sample.csv dan letakkan dalam variabel **data_tugas**.

--2024-03-25 16:06:37--  https://drive.google.com/file/d/1rJXZ3nu8I9cmzcBYLLIeqjXPAjXZQ_wY/view?usp=classroom_web
Resolving drive.google.com (drive.google.com)... 142.251.2.100, 142.251.2.113, 142.251.2.102, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.100|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized

Username/Password Authentication Failed.


In [44]:
import pandas as pd
data_tugas = pd.read_csv('/content/car_evaluation.csv')
data_tugas.head()

,buying,maint,lug_boot,safety,class
0,vhigh,vhigh,small,low,unacc
1,vhigh,vhigh,small,med,unacc
2,vhigh,vhigh,small,high,unacc
3,vhigh,vhigh,med,low,unacc
4,vhigh,vhigh,med,med,unacc


Bagi data menjadi data latih dan uji dengan nama **tugas_latih** dan **tugas_uji** dengan rasio 70%:30%

In [45]:
from sklearn.model_selection import train_test_split
tugas_latih, tugas_uji = train_test_split(data_tugas, test_size = 0.3)

Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya **tugas_label_uji**

In [46]:
tugas_label_uji = tugas_uji.pop('class')

Hitunglah nilai prior dari data latih dan simpan nilainya pada variabel bernama **prior_tugas**. Anda dapat menggunakan fungsi **hitung_prior** yang telah Anda buat.

In [48]:
prior_tugas = hitung_prior(tugas_latih['class'])
print(prior)

Counter({'Setosa': 0.3333333333333333, 'Virginica': 0.3333333333333333, 'Versicolor': 0.3333333333333333})


Buatlah fungsi bernama **hitung_likelihood_multinomial** dengan parameter input berupa data latih. Fungsi ini bertujuan menghitung nilai likelihood pada masing-masing kolom pada data latih. Operasi-operasi yang perlu dilakukan pada fungsi ini adalah:


1.   Mendapatkan daftar kolom data
2.   Mendapatkan nama kolom kelas pada data (kolom terakhir pada data)
3.   Mendapatkan nama-nama kelas pada data (unik).
4.   Menghitung likelihood pada masing-masing kolom, nilai kolom, dan nilai kelas

Sebagai saran, Anda dapat menggunakan struktur data dictionary untuk menyimpan nilai likelihood. Key pada dictionary tersebut adalah tuple(nama kolom, nilai pada kolom, nama kelas). Petunjuk penggunaan dictionary dengan key berupa tuple dapat dibaca [di sini](https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key).



In [49]:
def hitung_likelihood_multinomial(data_latih):
  list_columns = data_latih.columns[:-1]
  class_column_name = data_latih.columns[-1]
  list_class = set(data_latih[class_column_name])
  likelihood = {}
  hasil = {}
  for column in list_columns:
      likelihood[column] = {}
      for column_value in data_latih[column].unique():
          likelihood[column][column_value] = {}
          for a_class in list_class:
              count_wc = len(data_latih[(data_latih[column] == column_value) & (data_latih[class_column_name] == a_class)])
              count_c = len(data_latih[data_latih[class_column_name] == a_class])
              v = len(data_latih[column].unique())
              likelihood[column][column_value][a_class] = (count_wc + 1) / (count_c + v)
              hasil[(column, column_value, a_class)] = likelihood[column][column_value][a_class]

  return hasil


Ujilah fungsi **hitung_likelihood_multinomial** dengan parameter input berupa variabel **tugas_latih**

In [50]:
likelihood = hitung_likelihood_multinomial(tugas_latih)
print(likelihood)

{('buying', 'med', 'unacc'): 0.21718931475029035, ('buying', 'med', 'vgood'): 0.44680851063829785, ('buying', 'med', 'good'): 0.28846153846153844, ('buying', 'med', 'acc'): 0.32075471698113206, ('buying', 'low', 'unacc'): 0.21602787456445993, ('buying', 'low', 'vgood'): 0.5106382978723404, ('buying', 'low', 'good'): 0.6730769230769231, ('buying', 'low', 'acc'): 0.18867924528301888, ('buying', 'high', 'unacc'): 0.2694541231126597, ('buying', 'high', 'vgood'): 0.02127659574468085, ('buying', 'high', 'good'): 0.019230769230769232, ('buying', 'high', 'acc'): 0.3132075471698113, ('buying', 'vhigh', 'unacc'): 0.29732868757259, ('buying', 'vhigh', 'vgood'): 0.02127659574468085, ('buying', 'vhigh', 'good'): 0.019230769230769232, ('buying', 'vhigh', 'acc'): 0.17735849056603772, ('maint', 'low', 'unacc'): 0.2218350754936121, ('maint', 'low', 'vgood'): 0.40425531914893614, ('maint', 'low', 'good'): 0.6346153846153846, ('maint', 'low', 'acc'): 0.2641509433962264, ('maint', 'med', 'unacc'): 0.21138

Buatlah fungsi bernama **training_naive_bayes_multinomial** dengan input berupa data latih. Fungsi ini bertujuan membentuk model yang berisi:


1.   Nilai prior semua kelas
2.   Nilai likelihood semua kolom, nilai kolom, dan kelas
3.   Daftar kelas pada data latih (unik)
4.   Daftar nama kolom pada data latih

Return value dari fungsi ini adalah **model**



In [51]:
def training_naive_bayes_multinomial(data_latih):
  class_column_name = data_latih.columns[-1]
  prior = hitung_prior(data_latih[class_column_name])
  likelihood = hitung_likelihood_multinomial(data_latih)
  list_class = set(data_latih[class_column_name])
  list_columns = data_latih.columns[:-1]
  model = {}
  model['prior'] = prior
  model['likelihood'] = likelihood
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

Jalankan fungsi **training_naive_bayes_multinomial** dan simpan hasilnya dalam variabel **model_tugas**

In [52]:
model_tugas = training_naive_bayes_multinomial(tugas_latih)

Buatlah fungsi **testing_naive_bayes_multinomial** untuk mendapatkan kelas dari sebuah data uji. Langkah yang dilakukan hampir sama dengan Gaussian, yaitu mengalikan likelihood semua kolom dengan posterior.

In [53]:
def testing_naive_bayes_multinomial(model,data_uji):
  prior = model['prior']
  likelihood = model['likelihood']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class, 1)
  for a_class in list_class:
    for column in list_columns:
      column_value = data_uji[column]
      posterior[a_class] *= likelihood[(column, column_value, a_class)]
    posterior[a_class] *= prior[a_class]
  kelas_uji = max(posterior,key=posterior.get)
  return kelas_uji

Lakukan pengujian menggunakan sebuah data uji

In [54]:
idx = 10
uji = tugas_uji.iloc[idx]
kelas_uji = testing_naive_bayes_multinomial(model_tugas,uji)
print(kelas_uji)
print(tugas_label_uji.iloc[idx])

acc
acc


Lakukan pengujian terhadap semua data uji.

In [55]:
prediksi_total =[]
for indeks in range(tugas_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_multinomial(model_tugas,tugas_uji.iloc[indeks]))
print("Total prediksi benar: ",sum(prediksi_total==tugas_label_uji))

Total prediksi benar:  359


In [56]:
len(tugas_label_uji)

519